In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression

In [2]:
hybrid_vars = [0,1, 2, 3, 4, -3, -2, -1]

In [3]:
hybrid_model = TimeSeriesGradientBoosting(model1 = LinearRegression(), model2 = XGBRegressor(), model1_variables = hybrid_vars)

In [4]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(drop='first'
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

hybrid_xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', hybrid_model)
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [5]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithms=[hybrid_xgb_pipe, sgd_pipe], algorithm_name=["Gradient boosted decision tree", "Linear regression"], online=[False, False])
    return thesis


In [6]:
thesis = full_run()

ValueError: Found input variables with inconsistent numbers of samples: [8, 1096]

In [ ]:
results = thesis.resultsToDF()

In [ ]:
results[["Algorithm","SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Algorithm","Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                                      mean   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             3.555000   
                                                                Small             3.850000   
                                                 Half observed  Large            14.912500   
                                                                Small             6.245833   
                                                 Unobserved     Large            12.652500   
                                                                Small             5.828333   
                               No Drift          No Drift       No Drift          3.982500   
                               Sudden Drift      Fully observed Large             3.500000   
                                                                Small             3.785000   
                                                 Half observed  Large             4.765833   
                                                                Small             4.357500   
                                                 Unobserved     Large            16.534167   
                                                                Small             6.508333   
                               Sudden Shock      3 days         Large             4.155000   
                                                                Small             4.015000   
                                                 7 days         Large             4.220000   
                                                                Small             3.982500   
Linear regression              Incremental Drift Fully observed Large             5.060000   
                                                                Small             3.653333   
                                                 Half observed  Large            18.882500   
                                                                Small             6.267500   
                                                 Unobserved     Large            15.704167   
                                                                Small             5.703333   
                               No Drift          No Drift       No Drift          3.457500   
                               Sudden Drift      Fully observed Large             5.195000   
                                                                Small             3.613333   
                                                 Half observed  Large             7.402500   
                                                                Small             3.932500   
                                                 Unobserved     Large            19.900000   
                                                                Small             6.700000   
                               Sudden Shock      3 days         Large             3.625000   
                                                                Small             3.525000   
                                                 7 days         Large             3.607500   
                                                                Small             3.480000   

                                                                                           \
                                                                                      std   
Algorithm                      Drift type        Drift time     Drift magnitude             
Gradient boosted decision tree Incremental Drift Fully observed Large            0.284237   
                                                                Small            0.151598   
                                                 Half observed  Large            7.688757   
                                                

In [ ]:
results.sort_values("SMAPE", ascending=False)   

,Algorithm,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
236,Linear regression,0.0,Incremental Drift,Large,Medium,Half observed,1967.599593,30.33
219,Linear regression,0.0,Sudden Drift,Large,Medium,Unobserved,1096.013194,27.81
318,Linear regression,6.0,Incremental Drift,Large,Medium,Half observed,1382.909703,26.54
301,Linear regression,6.0,Sudden Drift,Large,Medium,Unobserved,866.403390,25.27
52,Gradient boosted decision tree,0.0,Sudden Drift,Large,Important,Unobserved,1022.348231,25.01
...,...,...,...,...,...,...,...,...
50,Gradient boosted decision tree,0.0,Sudden Drift,Large,Important,Fully observed,18.500273,3.23
132,Gradient boosted decision tree,6.0,Sudden Drift,Large,Important,Fully observed,17.548693,3.22
296,Linear regression,6.0,Sudden Drift,Large,Important,Fully observed,18.718341,3.21
109,Gradient boosted decision tree,3.0,Incremental Drift,Large,Important,Fully observed,17.959962,3.18
